In [30]:
from evaluator import GSM8kEvaluator
import re

In [2]:
evaluator = GSM8kEvaluator()

In [17]:
completion: str = '<reasoning>\nWeng earns $12 an hour for babysitting, and there are 60 minutes in an hour. \nShe just did 50 minutes of babysitting. \nTo calculate her earnings, we can divide the total time she worked by the average rate per hour: 50 minutes / $12/ hour = 4.1667 hours.\nSince she earns $12 an hour, to find her earnings for the 4.1667 hours, we multiply the number of hours by her hourly rate: 4.1667 hours * $12/hour = $49.996.\nSo, Weng earned $49.996 yesterday.\n</reasoning>\n<answer>\nWeng earned $49.996 from babysitting yesterday.'

In [18]:
formatted_completions = [[{"content": completion}]]

In [24]:
# Format the completion in the expected structure
evaluator._xml_count_reward(formatted_completions)

[-0.15500000000000003]

In [25]:
# try other ones
evaluator._soft_format_reward(formatted_completions)

[0.0]

In [20]:
# modified completion
modified_completion: str = '<reasoning>\nWeng earns $12 an hour for babysitting, and there are 60 minutes in an hour. \nShe just did 50 minutes of babysitting. \nTo calculate her earnings, we can divide the total time she worked by the average rate per hour: 50 minutes / $12/ hour = 4.1667 hours.\nSince she earns $12 an hour, to find her earnings for the 4.1667 hours, we multiply the number of hours by her hourly rate: 4.1667 hours * $12/hour = $49.996.\nSo, Weng earned $49.996 yesterday.\n</reasoning>\n<answer>\nWeng earned $49.996 from babysitting yesterday.\n</answer>\n'
formatted_modified_completions = [[{"content": modified_completion}]]

In [22]:
# Format the completion in the expected structure
evaluator._xml_count_reward(formatted_modified_completions)

[0.5]

In [23]:
evaluator._soft_format_reward(formatted_modified_completions)

[0.0]

In [26]:
evaluator._strict_format_reward(formatted_modified_completions)

[0.0]

Still fails -- due to newlines!

In [35]:
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

In [36]:
SYSTEM_PROMPT

'\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n'

In [34]:
# Try relaxed versions allowing newlines!
def new_strict_format_reward(completions) -> list[float]:
    """Reward for strict XML format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [bool(re.match(pattern, r, flags=re.DOTALL)) for r in responses]
    return [0.5 if m else 0.0 for m in matches]



def new_soft_format_reward(completions) -> list[float]:
    """Reward for relaxed XML format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [bool(re.search(pattern, r, flags=re.DOTALL)) for r in responses]
    return [0.5 if m else 0.0 for m in matches]



In [37]:
new_soft_format_reward(formatted_modified_completions)

[0.5]

In [38]:
new_strict_format_reward(formatted_modified_completions)

[0.5]